# 自然言語処理

項目

 1. 形態素解析
 2. 単語のベクトル化
 3. 文のベクトル化

## 形態素解析

日本語文字列を単語列に分割する処理です。形態素解析器Mecabがよく使われており、
新語の解析が必要な場合は、追加の辞書としてNeologd(https://github.com/neologd/mecab-ipadic-neologd)が
使われています。

### Mecabのインストール (Windows)

1. Microsoft Visual C++ 2008 SP1 Redistributable Package x86
のインストール
https://www.microsoft.com/en-us/download/details.aspx?id=5582

1. MeCab 64bitのインストール (標準の場所に)
https://github.com/ikegami-yukino/mecab/releases

1. インストール後のMecab\binにPATHを通す

1. パッケージのダウンロード
https://pypi.python.org/pypi/mecab-python-windows

1. Anaconda Promptを開き、MeCabパッケージの場所で
pip install mecab-python-windows





In [1]:
import MeCab

In [2]:
m = MeCab.Tagger("-Ochasen")
sent = "機械学習を要素技術としてもつ製品やサービスの設計・実装・評価ができる技術者の育成を目的とします。"
print(m.parse(sent))

機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
を	ヲ	を	助詞-格助詞-一般		
要素	ヨウソ	要素	名詞-一般		
技術	ギジュツ	技術	名詞-一般		
として	トシテ	として	助詞-格助詞-連語		
もつ	モツ	もつ	動詞-自立	五段・タ行	基本形
製品	セイヒン	製品	名詞-一般		
や	ヤ	や	助詞-並立助詞		
サービス	サービス	サービス	名詞-サ変接続		
の	ノ	の	助詞-連体化		
設計	セッケイ	設計	名詞-サ変接続		
・	・	・	記号-一般		
実装	ジッソウ	実装	名詞-サ変接続		
・	・	・	記号-一般		
評価	ヒョウカ	評価	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
できる	デキル	できる	動詞-自立	一段	基本形
技術	ギジュツ	技術	名詞-一般		
者	シャ	者	名詞-接尾-一般		
の	ノ	の	助詞-連体化		
育成	イクセイ	育成	名詞-サ変接続		
を	ヲ	を	助詞-格助詞-一般		
目的	モクテキ	目的	名詞-一般		
と	ト	と	助詞-格助詞-一般		
し	シ	する	動詞-自立	サ変・スル	連用形
ます	マス	ます	助動詞	特殊・マス	基本形
。	。	。	記号-句点		
EOS



単語分割の情報だけが必要な場合は、以下の引数で解析器を呼び出します。

In [3]:
m = MeCab.Tagger("-Owakati")
print(m.parse(sent).split())

['機械', '学習', 'を', '要素', '技術', 'として', 'もつ', '製品', 'や', 'サービス', 'の', '設計', '・', '実装', '・', '評価', 'が', 'できる', '技術', '者', 'の', '育成', 'を', '目的', 'と', 'し', 'ます', '。']


### word2vecを用いた単語の数値ベクトル化

最も単純に単語を特徴ベクトルに変換する方法はone-hot encodingです。
しかしこの方法では、(1)特徴ベクトルの次元数が単語の種類数となり、大きくなりすぎる。(2)全ての単語の距離が等距離となり、単語の意味の近さが表現できないという問題点があります。

そこで、一般に数万次元からなるone-hotベクトルを100～200次元程度の密な（値を持つ次元の数が多い）ベクトルに変換し、情報圧縮を行います。この操作をword2vecと呼びます。

参考サイト  
[Word2Vec：発明した本人も驚く単語ベクトルの驚異的な力](https://deepage.net/bigdata/machine_learning/2016/09/02/word2vec_power_of_word_vector.html)

### 準備

1. Anaconda Navigatorからgensimをインストール

2. word2vecのWikipediaでの学習済みモデルをダウンロード・展開し、entity_vector.model.binファイルをカレントディレクトリに置いておきます。
http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/

In [4]:
import warnings
warnings.filterwarnings('ignore')
from gensim.models import word2vec

## [KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html)の使い方

単語とベクトルの対応を記録しておけるデータ構造であるKeyedVectorsを用います。

In [5]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('entity_vector.model.bin', binary=True)

単語ベクトルの空間で、意味の近いものを求めてみます。

In [6]:
model.most_similar('台風')

[('高波', 0.7870876789093018),
 ('大雨', 0.7866235375404358),
 ('[台風]', 0.7853839993476868),
 ('強風', 0.7827271223068237),
 ('豪雨', 0.780258297920227),
 ('暴風雨', 0.780162513256073),
 ('集中豪雨', 0.7598928213119507),
 ('暴風', 0.7581937313079834),
 ('[集中豪雨]', 0.7475163340568542),
 ('[大雨]', 0.7466163635253906)]

ベクトルの引き算を行うことで、関係を抽出することができます。たとえば、「パリ」から「フランス」を引き算することで「首都」という関係が得られたと考えられるので、「日本」にこの「首都」という関係を足すと、「東京」が得られると期待できます。

In [7]:
model.most_similar(positive=['パリ','日本'], negative=['フランス'])

[('[日本]', 0.6431803703308105),
 ('東京', 0.6326844692230225),
 ('[東京]', 0.6148374080657959),
 ('日本国内', 0.5847295522689819),
 ('大阪', 0.5656222105026245),
 ('都内', 0.5597203373908997),
 ('東京都内', 0.5482653975486755),
 ('[大阪]', 0.5426324605941772),
 ('神戸', 0.520367443561554),
 ('[横浜]', 0.5199853181838989)]

In [8]:
model.most_similar(positive=['たこ焼き','香川'], negative=['大阪'])

[('焼き肉', 0.5954942107200623),
 ('うどん', 0.5942089557647705),
 ('コロッケ', 0.5901445150375366),
 ('牛丼', 0.5884590148925781),
 ('カツ', 0.5824934840202332),
 ('ネギ', 0.5817806720733643),
 ('おでん', 0.5800793170928955),
 ('パフェ', 0.5788909196853638),
 ('おにぎり', 0.5777252912521362),
 ('焼肉', 0.5764982104301453)]

### Doc2Vec

文や文章をベクトル化して、類似度計算を可能にします。

参考サイト  
[Doc2Vecの仕組みとgensimを使った文書類似度算出チュートリアル](https://deepage.net/machine_learning/2017/01/08/doc2vec.html)

In [22]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

m = MeCab.Tagger("-Owakati")

sentences = [
    "２時にアラームをセット",
    "アラームを３時に設定",
    "５時になったら知らせて",
    "アラームを使いたい",
    "３０分後に起こして",
    "２時に京都に着きたい",
    "乗換案内を起動",
    "京都から東京までの新幹線",
    "東京行きの最終電車",
]

# 空のリストを作成（学習データとなる各文書を格納）
X = []
for s, t in zip(sentences, range(len(sentences))):
    X.append(TaggedDocument(words=m.parse(s).split(), tags=[t]))   

# 学習
# documents:学習データ（TaggedDocumentのリスト）
# min_count: 学習に使用する単語の最低出現回数
# dm:学習モデル 0:DBOW, 1:DM（デフォルト)
model = Doc2Vec(documents=X, min_count=1, dm=1)
 
# 学習したモデルを保存
model.save('doc2vec.model')

# 検索
model.docvecs.most_similar([2])

[(1, 0.2527516186237335),
 (3, 0.2247992753982544),
 (4, 0.15878286957740784),
 (8, 0.06908675283193588),
 (7, 0.04946136474609375),
 (5, -0.005576267838478088),
 (6, -0.006039753556251526),
 (0, -0.1989622712135315)]